In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pygad
from importlib import reload
from multiprocessing import Pool
import pandas as pd 

import tgs 
import rgs 
import key_rate_optimization as opt

reload(tgs)
reload(rgs)
reload(opt)

<module 'key_rate_optimization' from '/Users/gordonwongyl/Documents/ORF/Reproduce_result/key_rate_optimization.py'>

In [ ]:
# Set the paramaters for optimization
# Values in Table 2
GAMMA = np.array([2e9, 100e9, 170e6, 100e9]) * 2 * np.pi
T_SPIN_COHERENCE = [13e-3, 4e-6, 1., 1.]
routines = [opt.tgs_a_routine, opt.tgs_f_routine, opt.rgs_a_routine, opt.rgs_f_routine]
inputs = [{"rout": rout, "GAMMA": GAMMA[i], "T_SPIN_COHERENCE": T_SPIN_COHERENCE[i], "L": 1000e3} for i in range(4) for rout in routines]


with Pool() as pool:
    result = pool.map(opt.optimization, inputs)

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(20, 20))
routines = [opt.tgs_a_routine, opt.tgs_f_routine, opt.rgs_a_routine, opt.rgs_f_routine]

for name_in in range(4):
    name = routines[name_in].__class__.__name__
    for param_in in range(4):
        for iter in range(3):   
            ga_instance = pygad.load("optimized_result/" + name + f"_gamma_{GAMMA[param_in]}_spin_coh_{T_SPIN_COHERENCE[param_in]}_L_{1000e3}_iter_{iter+1}")
            y_data = ga_instance.best_solutions_fitness
            x_data = list(range(1, len(y_data)+1))
            ax[param_in][name_in].plot(x_data, y_data, label=f"{iter+1}. {ga_instance.best_solution()[0][:-1]}")
            ax[param_in][name_in].set_title(name)
            # ax[name_in][param_in].legend()
fig.show()

In [39]:
for name_in in range(4):
    name = scheme_list[name_in]
    for param_in in range(4):
        for iter in range(3):   
            ax[param_in][name_in].legend()
fig.savefig('output.png', facecolor='white')

In [ ]:
GAMMA = 10**np.arange(7,13)*2*np.pi
T_SPIN_COHERENCE = 10.**np.arange(-7,3)
# routines = [opt.tgs_a_routine, opt.tgs_f_routine, opt.rgs_a_routine, opt.rgs_f_routine]
routines = [opt.tgs_a_routine]
inputs = [{"rout": rout, "GAMMA": g, "T_SPIN_COHERENCE": t, "L": 1000e3} for g in GAMMA for t in T_SPIN_COHERENCE for rout in routines]
with Pool() as pool:
    result = pool.map(opt.optimization, inputs)

In [ ]:
L_list = np.linspace(1e3,3000e3,20)
GAMMA = np.array([2e9, 100e9, 170e6, 100e9]) * 2 * np.pi
T_SPIN_COHERENCE = [13e-3, 4e-6, 1., 1.]
routines = [opt.tgs_a_routine, opt.tgs_f_routine, opt.rgs_a_routine, opt.rgs_f_routine]
inputs = [{"rout": rout, "GAMMA": GAMMA[i], "T_SPIN_COHERENCE": T_SPIN_COHERENCE[i], "L": L} for rout in routines for i in range(4) for L in L_list ]
with Pool() as pool:
    result = pool.map(opt.optimization, inputs)

In [11]:
cols_name = ['rout', 'GAMMA', 'T_SPIN_COHERENCE', 'L', 'best_solutions_fitness', 'best_solution']
import csv
with open('opt_record_3000e3_2D.csv', 'w') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = cols_name) 
    writer.writeheader() 
    writer.writerows(result)  
    